In [1]:
dir_map = {
    '^': (0, -1),
    'v': (0, 1),
    '<': (-1, 0),
    '>': (1, 0)
}

In [2]:
class Box():
    def __init__(self, start_pos, width=1):
        if width > 2 or width < 1:
            raise ValueError("Width must be 1 or 2")
        self.pos = start_pos
        self.width = width

    def __repr__(self):
        return f"O" if self.width == 1 else "[]"
    
    def move(self, direction, doc):
        x, y = self.pos
        nx, ny = (x + dir_map[direction][0], y + dir_map[direction][1])
        doc[y][x] = '.'
        self.pos = (nx, ny)
        doc[ny][nx] = self
        if width == 2:
            if direction != '>':
                doc[y][x + 1] = '.'
            doc[ny][nx + 1] = ']'


    def can_be_pushed(self, direction, doc, boxes=[]):
        nx, ny = (self.pos[0] + dir_map[direction][0], self.pos[1] + dir_map[direction][1])
        new_obj = doc[ny][nx]
        if nx < 0 or  ny < 0:
            return False, False
        if new_obj == '#':
            return False, False
        if type(new_obj) == Box or new_obj == ']':
            if direction == '>':
                new_obj = doc[ny][nx + 1]
            if new_obj == ']':
                new_obj = doc[ny][nx - 1]
            if type(new_obj) == Box:
                can_move, boxes = new_obj.can_be_pushed(direction, doc, boxes)              
        if new_obj == '.':
            can_move = True
        # Don't need to check left because we are the right side of the box
        if self.width == 2 and direction != '<':
            nx += 1
            right_obj = doc[ny][nx]
            if right_obj == '#':
                return False, False
            if type(right_obj) == Box or right_obj == ']':
                if right_obj == ']':
                    right_obj = doc[ny][nx - 1]               
                can_move, boxes = right_obj.can_be_pushed(direction, doc, boxes)              
            if right_obj == '.':
                can_move = True

        if not can_move or boxes == False:
            return False, False
        boxes.append(self)
        return can_move, boxes
      
    
class Robot():
    def __init__(self, start_pos, moves):
        self.pos = start_pos
        self.moves = moves

    def __repr__(self):
        return f"@"
    
    def move(self, direction, doc):
        x, y = self.pos
        nx, ny = (x + dir_map[direction][0], y + dir_map[direction][1])
        obj = doc[ny][nx]
        if obj == '#':
            return
        if type(obj) == Box or obj == ']':
            if obj == ']':
                obj = doc[ny][nx - 1]
            can_move, boxes_to_update = obj.can_be_pushed(direction, doc, [])
            if not can_move:
                return

            moved_boxes = set()
            for box in boxes_to_update:
                if box not in moved_boxes:
                    box.move(direction, doc)
                    moved_boxes.add(box)
        self.pos = (nx, ny)
        doc[y][x] = '.'
        doc[ny][nx] = self

def pretty_print_doc(doc):
    to_print = "\n".join(["".join([str(cell) for cell in row]) for row in doc])
    to_print = to_print.replace(']]', ']')
    print(to_print)


all_boxes = []
width = 2

# with open("small_test.txt") as fh:
# with open("test_input.txt") as fh:
with open("input.txt") as fh:
    input = fh.read()
    doc, moves = input.split("\n\n")
    moves = list(''.join([x.strip() for x in moves]))
    doc = [list(x) for x in doc.split("\n")]
    if width == 2:
        new_doc = []
        for row in doc:
            new_row = []
            for cell in row:
                if cell == '#':
                    new_row.append('#')
                    new_row.append('#')
                    continue
                if cell == 'O':
                    new_row.append('[')
                    new_row.append(']')
                    continue
                elif cell == '.':
                    new_row.append('.')
                elif cell == '@':
                    new_row.append('@')
                new_row.append('.')
            new_doc.append(new_row)
        doc = new_doc

    for y, row in enumerate(doc):
        for x, cell in enumerate(row):
            if cell == '@':
                robot = Robot((x, y), moves)
                doc[y][x] = robot
            elif cell == 'O' or cell == '[':
                box = Box((x, y), width=width)
                doc[y][x] = box
                all_boxes.append(box)
print(len(doc), len(doc[0]))
pretty_print_doc(doc)


for m in robot.moves:
    # print(m)
    robot.move(m, doc)
    # pretty_print_doc(doc)

tot = 0
for box in all_boxes:
    tot += 100 * box.pos[1] + box.pos[0]

print(tot)



50 100
####################################################################################################
##[]..[]..[]..[]..............[]..##[]......[]..[][]..[]..##....[]....[]..........[][]..[]........##
##[]..[]..[]....[]..[][][]..[]............[]....[][]....##[]..[]........[]..[]##....[]....[][]....##
##..[]..[]........[][]..[][]..........[][][]..............[]........................[][]..[]......##
##..............[]..##..[][]....##[]..[]........[]....[][]..[]....................[]##....[]....####
##........[]......[]..........[]##........[]..[]......##..[]..[]....[]....##........##..........####
##..[]..##..[]....[]..[][]..[]..[]..[][]....[]##....##[]......[]####........##..[]..[]....[]....[]##
##....[][]..####[]....[]........##..[][][]..[]##[]....[]......##..[]....[]..[]..[][]....[][]......##
##..##............[]####........[]..[]....[]........[]....[][][][]............[]..##[]##......##..##
##..............##[]..##........[][]........[]........[][]........[][]##..##........